# Research Question:
### What are the most influential variables on the severity of accidents?

Useful Paper:
    https://www.sciencedirect.com/science/article/pii/S2590198223000611

In [1]:
#Display Spark Output in scrollable format within jupyter notebook
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
#Supress Warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

# Load Data

In [4]:
#Instantiate Spark Session
spark = (SparkSession
  .builder
  .appName("US_Accidents")
  .getOrCreate())
spark.sparkContext.setLogLevel("ERROR") #supress warnings

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 13:21:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Read in Dataset
df = spark.read.parquet("final_dataset.parquet")
df.show(5)

+--------+--------------+-----------+------------+--------------+---------------+-----------------+-------+---------+-------+----+----+-------------+--------------------------+--------------------+---------------------------------+-----------------+-----------------+-------------------+------------+--------------------------+-----------+--------------------+--------------------+-----------+
|Severity|Temperature(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Speed(mph)|Precipitation(in)|Weekday|Rush Hour|Holiday|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate Indicator|Sex ratio (males per 100 females)|Percent_Age_15-19|Percent_Age_20-24|Percent_Age_65_over|MedianIncome|MedianIncome_MarginOfError|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|
+--------+--------------+-----------+------------+--------------+---------------+-----------------+-------+---------+-------+----+----+-------------+--------------------------+--------------------+---------------

In [6]:
# Remove MedianIncome_MarginOfError
df = df.drop('MedianIncome_MarginOfError')
df.show(5)

+--------+--------------+-----------+------------+--------------+---------------+-----------------+-------+---------+-------+----+----+-------------+--------------------------+--------------------+---------------------------------+-----------------+-----------------+-------------------+------------+-----------+--------------------+--------------------+-----------+
|Severity|Temperature(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Speed(mph)|Precipitation(in)|Weekday|Rush Hour|Holiday|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate Indicator|Sex ratio (males per 100 females)|Percent_Age_15-19|Percent_Age_20-24|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|
+--------+--------------+-----------+------------+--------------+---------------+-----------------+-------+---------+-------+----+----+-------------+--------------------------+--------------------+---------------------------------+-----------------+-----------------

In [7]:
# Get row count
rows = df.count()
print(f"DataFrame Rows count : {rows}")

# Get columns count
cols = len(df.columns)
print(f"DataFrame Columns count : {cols}")

DataFrame Rows count : 7026806
DataFrame Columns count : 24


# Sampling

In [8]:
# Check Class Imbalance
cts = df.groupBy("Severity").count().withColumn('percent', (F.col('count') / rows)*100)
cts.show()

+--------+-------+------------------+
|Severity|  count|           percent|
+--------+-------+------------------+
|       1|  65142|0.9270499285165977|
|       3|1123799|15.993027272988611|
|       4| 178821|2.5448404296347444|
|       2|5659044| 80.53508236886005|
+--------+-------+------------------+



# Underperforming Tests Overview:

## Tried Random split (80/20)
- As predicted, model predicted severity = 2 for everything and they accuracy = the % of test samples with label = 2

train_data, test_data = df.randomSplit([0.8, 0.2])

## Undersampling (80/20) - undersampled each class by the smallest class for the training set - testing set everything else
- This decreased the training set to about 3.5% of the original data set and the accuracy (~32% reflected that)
- With undersampling, the performance might change with the sample taken

sample = (cts.select("count").rdd.min()[0])*0.8 #Undersample each class by 80% of the smallest class

class1 = sample/(cts.select("count").where(cts.Severity == '1').rdd.min()[0])

class2 = sample/(cts.select("count").where(cts.Severity == '2').rdd.min()[0])

class3 = sample/(cts.select("count").where(cts.Severity == '3').rdd.min()[0])

class4 = sample/(cts.select("count").where(cts.Severity == '4').rdd.min()[0])

df.createOrReplaceTempView("data_view") #Create a temporary view to use SQL

fractions = {1: class1, 2: class2, 3: class3, 4: class4} #downsample each class to 80% of the smallest class

train_data = df.sampleBy("Severity", fractions, seed=42) #Use stratified sampling to maintain class distribution

test_data = df.subtract(train_data)

## Did not try full Oversampling
- I'm always wary of oversampling amplyfying outliers especially with such high class imbalance and with regression trees that are already prone to overfitting
## A combination of oversampling and undersampling may be a nice compramise
- Did not check

## Binary Classification (1|2 = 0, 3|4 = 1)
- Best so far - model f1 score ~ 0.77 with base rf model - will obviously be better as half the options to guess from but still major improvement

features = df.select([col for col in df.columns if col != "Severity" and col != "Severity_Binary"]).columns # Select all features except target variable

assembler = VectorAssembler(inputCols=features, outputCol='features') # Vectorize Features

model = RandomForestClassifier(featuresCol = 'features', labelCol = 'Severity_Binary') # Model
  
pipeline = Pipeline(stages=[assembler, model]) # Creating the pipeline 

fit_model = pipeline.fit(train_data) #train

results = fit_model.transform(test_data) #predict

## Binary Classification

In [9]:
# Binary Classification (1 or 2 vs 3 or 4)
df = df.withColumn('Severity_Binary', when((col("Severity")==1) | (col("Severity")==2), 0).otherwise(1))
df = df.drop('Severity')
df.show(5)

+--------------+-----------+------------+--------------+---------------+-----------------+-------+---------+-------+----+----+-------------+--------------------------+--------------------+---------------------------------+-----------------+-----------------+-------------------+------------+-----------+--------------------+--------------------+-----------+---------------+
|Temperature(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Speed(mph)|Precipitation(in)|Weekday|Rush Hour|Holiday|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate Indicator|Sex ratio (males per 100 females)|Percent_Age_15-19|Percent_Age_20-24|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Severity_Binary|
+--------------+-----------+------------+--------------+---------------+-----------------+-------+---------+-------+----+----+-------------+--------------------------+--------------------+---------------------------------+-----------------+------------

In [11]:
# Check Class Imbalance
cts = df.groupBy("Severity_Binary").count().withColumn('percent', (F.col('count') / rows)*100)
cts.show()

+---------------+-------+------------------+
|Severity_Binary|  count|           percent|
+---------------+-------+------------------+
|              1|1302620|18.537867702623352|
|              0|5724186| 81.46213229737664|
+---------------+-------+------------------+



In [12]:
# Undersample each class by 80% of the smallest class
sample = (cts.select("count").rdd.min()[0])*0.8

class0 = sample/(cts.select("count").where(cts.Severity_Binary == '0').rdd.min()[0])
class1 = sample/(cts.select("count").where(cts.Severity_Binary == '1').rdd.min()[0])

# Split Data by Class - Downsampling

# Create a temporary view to use SQL
df.createOrReplaceTempView("data_view")

# Calculate fractions for each class
#fractions = df.groupBy("Severity").count().rdd.map(lambda row: (row[0], 0.8)).collectAsMap() #samples 80% of each class
fractions = {0: class0, 1: class1} #downsample each class to 80% of the smallest class

# Use stratified sampling to maintain class distribution
train_data = df.sampleBy("Severity_Binary", fractions, seed=42)
test_data = df.subtract(train_data)

In [13]:
# Print Overall % Sampled from DF
print(train_data.count()/df.count()*100)

# Print % Sampled for each class within Train Data
train_data.groupBy("Severity_Binary").count().withColumn('percent', (F.col('count') / train_data.count())*100).show()

29.67624266274037
+---------------+-------+------------------+
|Severity_Binary|  count|           percent|
+---------------+-------+------------------+
|              1|1041869|49.962739031272356|
|              0|1043423| 50.03726096872764|
+---------------+-------+------------------+



In [14]:
# Print Overall % Sampled from DF
print(test_data.count()/df.count()*100)

# Print % Sampled for each class within Train Data
test_data.groupBy("Severity_Binary").count().withColumn('percent', (F.col('count') / test_data.count())*100).show()

54.54452563511786


+---------------+-------+-----------------+
|Severity_Binary|  count|          percent|
+---------------+-------+-----------------+
|              1| 233625|6.095511876888011|
|              0|3599113|93.90448812311199|
+---------------+-------+-----------------+



# Modeling

In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Select all features except target variable
feature_list = []
for col in df.columns:
    if col == 'Severity_Binary':
        continue
    else:
        feature_list.append(col)

# Vectorize Features
assembler = VectorAssembler(inputCols=feature_list, outputCol='features')

# Create a RandomForestClassifier
rf = RandomForestClassifier(labelCol="Severity_Binary", featuresCol="features")

# Create a pipeline
pipeline = Pipeline(stages=[assembler, rf])

# Define the parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [3, 6, 8]) \
    .addGrid(rf.maxDepth, [3, 4, 5]) \
    .build()

# Create a CrossValidator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="Severity_Binary", predictionCol="prediction"),
                          numFolds=5)

# Fit the model
cvModel = crossval.fit(train_data)

# Make predictions on the test set
predictions = cvModel.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="Severity_Binary", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)
print("F1: ", f1)

# Get the best model
bestModel = cvModel.bestModel
print(bestModel.stages[-1]._java_obj.paramMap()) 

KeyboardInterrupt: 

In [28]:
# Further Metrics on Best Model
pred = cvModel.bestModel.transform(test_data)

#Evaluate (Confusion Matrix, Accuracy, Weighted Precision, Recall, and F1 Score)
predictionAndLabels = pred.select("prediction", "Severity_Binary")
metrics = MulticlassMetrics(predictionAndLabels.rdd.map(lambda x: tuple(map(float, x))))

# Get confusion matrix
print(metrics.confusionMatrix().toArray())

# Get precision, recall, and F1-score for each class
print(f'Weighted Precision: {metrics.weightedPrecision}') #would expect to be good when test sample has high majority 0 class
print(f'Weighted Recall: {metrics.weightedRecall}')
print(f'Weighted F1 Score: {metrics.weightedFMeasure()}') #would like to optimize this (balance of precision and recall)
print(f'Accuracy: {metrics.accuracy}') #could be skewed with imbalanced test set

[[2855213.  743900.]
 [ 115948.  117677.]]
Weighted Precision: 0.9107245963203416
Weighted Recall: 0.7756569846412669
Weighted F1 Score: 0.829251596545379
Accuracy: 0.7756569846412669


In [ ]:
# ROC Curve - he asked for ROC curve

In [36]:
#Feature Importances
bestModel = cvModel.bestModel.stages[-1]  # Assuming the last stage in the pipeline is the model

featureImportances = bestModel.featureImportances

print("Feature Importances:")
for feature, importance in zip(test_data.columns[:-1], featureImportances):
    print(f"{feature}: {importance}")

Feature Importances:
Temperature(F): 0.0009213015483506297
Humidity(%): 0.002540609439652656
Pressure(in): 0.020450333607135187
Visibility(mi): 0.0013161062441105088
Wind_Speed(mph): 0.0940699616214557
Precipitation(in): 0.0
Weekday: 0.0005017290970311592
Rush Hour: 0.0
Holiday: 0.0
Rain: 0.0023240584289040468
Snow: 0.0
SeasonVec: 0.0
Astronomical_TwilightIndex: 0.0
Interstate Indicator: 0.0
Sex ratio (males per 100 females): 0.0
Percent_Age_15-19: 0.7070260166982979
Percent_Age_20-24: 0.0002243347183741944
Percent_Age_65_over: 0.0
MedianIncome: 0.003956996597709727
Urban_Ratio: 0.0
Traffic_Interference: 0.01904514582410056
Traffic_Intersection: 0.0169767010785899
Destination: 0.0


In [31]:
bestModel.featureImportances #why are there 35?

SparseVector(25, {0: 0.0009, 1: 0.0025, 2: 0.0205, 3: 0.0013, 4: 0.0941, 6: 0.0005, 9: 0.0023, 15: 0.707, 16: 0.0002, 18: 0.004, 20: 0.019, 21: 0.017, 23: 0.1054, 24: 0.0253})